In [1]:
# Topgyal Gurung
# Decision Tree Algorithm from scratch on monk dataset
import pandas as pd
import numpy as np
from pprint import pprint

In [2]:
# load and prepare dataset
# train_set
monk1_train=pd.read_csv("/Users/topgyalgurung/Desktop/monk-dataset/train1.csv")
monk2_train=pd.read_csv("/Users/topgyalgurung/Desktop/monk-dataset/train2.csv")
monk3_train=pd.read_csv("/Users/topgyalgurung/Desktop/monk-dataset/train3.csv")
# test_set
monk1_test=pd.read_csv("/Users/topgyalgurung/Desktop/monk-dataset/test1.csv")
monk2_test=pd.read_csv("/Users/topgyalgurung/Desktop/monk-dataset/test2.csv")
monk3_test=pd.read_csv("/Users/topgyalgurung/Desktop/monk-dataset/test3.csv")
# header=None 

In [3]:
# drop Id 
monk1_train=monk1_train.drop("Id", axis=1)
monk2_train=monk2_train.drop("Id",axis=1)
monk3_train=monk3_train.drop("Id",axis=1)

monk1_test=monk1_test.drop("Id",axis=1)
monk2_test=monk2_test.drop("Id",axis=1)
monk3_test=monk3_test.drop("Id",axis=1)

In [4]:
# dataframe into numpy array
monk1_data=monk1_train.values
monk2_data=monk2_train.values
monk3_data=monk3_train.values

In [5]:
# base case 
def check_purity(data):
    label_column=data[:,0]
    unique_classes=np.unique(label_column)
    if len(unique_classes)==1:
        return True
    else:
        return False

In [6]:
def classify_data(data):
    label_column=data[:,0]
    unique_classes,counts_unique_classes=np.unique(label_column,return_counts=True) # [0 1] , [62 62]
    index=counts_unique_classes.argmax()  # use max class as index 
    classification=unique_classes[index]
    return classification

In [7]:
def get_potential_splits(data):
    potential_splits={}
    _,n_columns=data.shape  #iterate over column
    for column_i in range(1,n_columns):  # column index exclude class
        values = data[:, column_i]
        unique_values = np.unique(values)
        potential_splits[column_i]=unique_values
    return potential_splits  

In [8]:
# split data to data equal if equal and data_not for not equal
def split_data(data,split_column,split_value):
    #random.shuffle(data)
    split_column_values=data[:,split_column]
    true_data=data[split_column_values==split_value]
    false_data=data[split_column_values!=split_value]
    return true_data,false_data  

In [9]:
# entropy: measure of uncertainty
def entropy(data):
    label_column=data[:,0]
    _,counts=np.unique(label_column,return_counts=True)
    
    probability=counts/counts.sum()
    entropy=sum(probability*-np.log2(probability))
    return entropy

In [10]:
entropy(monk1_data)

1.0

In [11]:
entropy(monk2_data)

0.957117428264771

In [12]:
entropy(monk3_data)

0.999806132804711

In [13]:
def overall_entropy(true_data, false_data):
    n=len(true_data)+len(false_data)
    p_true_data=len(true_data)/n
    p_false_data=len(false_data)/n
    overall_entropy=(p_true_data * entropy(true_data) + p_false_data * entropy(false_data))
    return overall_entropy

In [14]:
# def overall_entropy(data, true_data, false_data):
#     p=len(true_data)/len(data)
#     overall_entropy=entropy(data) - p* entropy(true_data)-(1-p) *entropy(false_data)
#     return overall_entropy

In [15]:
def best_split(data,potential_splits):
    temp_entropy=999  #arbitary large num
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            true_data,false_data=split_data(data,split_column=column_index,split_value=value)
            #info_gain
            current_overall_entropy=overall_entropy(true_data,false_data)
            #print(current_overall_entropy)
            if current_overall_entropy<=temp_entropy:
                temp_entropy=current_overall_entropy
                best_split_column=column_index
                best_split_value=value
    return best_split_column, best_split_value

In [16]:
# recursive decision_tree algorithm
def decision_tree(train_set,counter=0):
    if counter==0:
        global column_headers
        column_headers=train_set.columns
        data=train_set.values  # numpy array
    else:
        data= train_set   # pandas dataframe
    
    # base case
    if check_purity(data):
        classification=classify_data(data)
        return classification
    # recursive
    else:
        counter+=1
        # run helper functions
        potential_splits=get_potential_splits(data)
        split_column,split_value=best_split(data,potential_splits)
        true_data,false_data=split_data(data,split_column,split_value)

        # sub_tree
        feature_name=column_headers[split_column]
        question="{} == {}".format(feature_name,split_value)
        sub_tree={question:[]} #empty list
        # find ans (recursive part)
        yes_ans=decision_tree(true_data,counter)
        no_ans=decision_tree(false_data,counter)
        
        if yes_ans==no_ans:
            sub_tree=yes_ans
        else:
            sub_tree[question].append(yes_ans)
            sub_tree[question].append(no_ans)

        return sub_tree

In [17]:
def classify_df(df,tree):
    question=list(tree.keys())[0]
    feature_name,operator,value=question.split(" ")
    
    if str(df[feature_name])== value:
        ans=tree[question][0]
    else:
        ans=tree[question][1]
    
    # base case
    if not isinstance(ans,dict):
        
        return ans
    # recursion
    else:
        residual_tree=ans
        return classify_df(df,residual_tree)

In [25]:
def accuracy(df,tree):
    df["classification"]=df.apply(classify_df,args=(tree,),axis=1)
    df["classification_correct"]=df["classification"]==df["class"]
    accuracy=df["classification_correct"].mean()
    return accuracy

###  Evaluation

In [26]:
monk1_tree=decision_tree(monk1_train)
monk2_tree=decision_tree(monk2_train)
monk3_tree=decision_tree(monk3_train)

#### MONK 1 

In [27]:
print("MONK-1 TREE:")
pprint(monk1_tree)

MONK-1 TREE:
{'a5 == 1': [1,
             {'a1 == 1': [{'a2 == 1': [1, 0]},
                          {'a2 == 1': [0,
                                       {'a5 == 3': [{'a2 == 3': [{'a1 == 3': [1,
                                                                              0]},
                                                                 {'a1 == 3': [0,
                                                                              1]}]},
                                                    {'a4 == 1': [{'a6 == 2': [{'a3 == 2': [1,
                                                                                           {'a5 == 4': [{'a2 == 3': [1,
                                                                                                                     0]},
                                                                                                        1]}]},
                                                                              1]},
                      

In [28]:
classify_df(monk1_train,monk1_tree)

1

In [29]:
classify_df(monk1_test,monk1_tree)

1

In [30]:
accuracy(monk1_train,monk1_tree)

1.0

In [31]:
accuracy(monk1_test,monk1_tree)

0.9004629629629629

## MONK 2

In [32]:
print("MONK-2 TREE:")
pprint(monk2_tree)

MONK-2 TREE:
{'a4 == 1': [{'a5 == 1': [{'a6 == 2': [{'a3 == 2': [{'a2 == 3': [1, 0]}, 0]},
                                       0]},
                          {'a1 == 1': [{'a6 == 2': [{'a3 == 2': [1, 0]}, 0]},
                                       {'a6 == 2': [{'a3 == 2': [{'a2 == 1': [1,
                                                                              0]},
                                                                 {'a2 == 1': [0,
                                                                              1]}]},
                                                    {'a3 == 2': [{'a2 == 1': [0,
                                                                              1]},
                                                                 0]}]}]}]},
             {'a5 == 2': [{'a3 == 2': [{'a2 == 2': [0,
                                                    {'a1 == 1': [1,
                                                                 {'a2 == 3': [0,
             

In [33]:
classify_df(monk2_train,monk2_tree)

1

In [34]:
classify_df(monk2_test,monk2_tree)

1

In [35]:
accuracy(monk2_train,monk2_tree)

1.0

In [36]:
accuracy(monk1_test,monk2_tree)

0.4861111111111111

## MONK -3 

In [37]:
print("MONK-3 TREE:")
pprint(monk3_tree)

MONK-3 TREE:
{'a2 == 3': [{'a4 == 1': [{'a3 == 2': [0, {'a1 == 1': [0, 1]}]}, 0]},
             {'a5 == 4': [0,
                          {'a5 == 3': [{'a3 == 2': [1,
                                                    {'a1 == 3': [1,
                                                                 {'a6 == 2': [{'a4 == 3': [{'a2 == 2': [{'a1 == 2': [0,
                                                                                                                     1]},
                                                                                                        1]},
                                                                                           1]},
                                                                              {'a4 == 1': [{'a2 == 2': [1,
                                                                                                        0]},
                                                                                           0]}]}]}]},


In [38]:
classify_df(monk3_train,monk3_tree)

1

In [39]:
classify_df(monk3_test,monk3_tree)

1

In [40]:
accuracy(monk1_train,monk3_tree)

0.5725806451612904

In [41]:
accuracy(monk1_test,monk3_tree)

0.5625

# Main function 

In [42]:
# if __name__=="__main__":
#     monk1_tree=decision_tree(monk1_train)
#     monk2_tree=decision_tree(monk2_train)
#     monk3_tree=decision_tree(monk3_train)
    
#     print("MONK-1 TREE:")
#     pprint(monk1_tree)
#     print("MONK-2 TREE:")
#     pprint(monk2_tree)
#     print("MONK-3 TREE:")
#     pprint(monk3_tree)

#     classify(monk1_train,monk1_tree)
#     accuracy(monk1_train,monk1_tree)

#     post-prune tree
    
#     classify(monk1_test,monk1_tree)
#     accuracy(monk1_test,monk1_tree)

